In [1]:
import re

import pandas as pd

from src.dataset import read_dataset, tokenize_dataset

In [2]:
# replace path with local dataset path
data = read_dataset("snippets-dev/snippets-dev.db", "Python")
data

,language,code
0,Python,"version = '7'\nhtml_title = ""Guzzle Documentat..."
1,Python,"# Path to a touch icon\n # ""touch_icon""..."
2,Python,"""base_url"": ""http://guzzlephp.org""\n\n ..."
3,Python,from tensorflow.python.estimator.model_fn impo...
4,Python,self.assertFalse(gfile.Exists('ram://exist...
...,...,...
196460,Python,"print(""brody2:"", brody2)\n\n cosDist = ..."
196461,Python,"\n def getJSCode(self):\n map = {""paths"": ..."
196462,Python,if not isPivot:\n simpleRing = shapel...
196463,Python,new_geometries.append( Geometry(shapely....


In [3]:
for i, language, code in data.sample(3).itertuples():
    print(f"###### Index {i:7} ######")
    print(code)
    print(f"###########################")

###### Index  138506 ######
    print("\t                                   * A           *")
    print("\t                                   *             *")
    print("\t                                   *      *      *")
    print("\t                                   *     ***     *")
    print("\t                                   *    *****    *")

###########################
###### Index   67705 ######
            80,
            27,
            2,
            7,
            97,

###########################
###### Index   16740 ######
                         idx + 1, self._thread_count, name)
            thread = FSThread(name=name,
                              target=self._target,
                              args=self._args,
                              kwargs=self._kwargs)

###########################


In [4]:
tokenize_dataset(data.sample(5))

c:\Python310\lib\site-packages\pandas\core\apply.py:853: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ares = np.asarray(res).ndim


,language,code
150738,Python,"[def, _SetUpProductReferences, (, self, other_..."
194545,Python,"["", "", "", \n, Hierarchical, Sigmoid, Layer, .,..."
56632,Python,"[\n, def, test_qs_exists_handles_value_error, ..."
167076,Python,"[formats, =, list, (, ctx, [, ', formats, ', ]..."
95044,Python,"[np, ., testing, ., assert_equal, (, list, (, ..."
